In [2]:
import os

try:
    import PyQt5.QtCore
    %matplotlib qt
except ImportError:
    %matplotlib inline
import keras
import mne
import numpy as np
import pandas as pd
from scipy.io import loadmat
import tensorflow as tf
import random

from mne.channels import make_standard_montage
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model


In [3]:
data_dir = os.path.dirname("./data/")
data_files = os.listdir(data_dir)

In [4]:
def annotations_from_eGUI(raw, egui):
    codes = []
    starts = []

    current_state = None

    for i in range(len(egui)):
        if egui[i][0] != current_state:
            starts.append(i)
            current_state = egui[i][0]
            codes.append(str(egui[i][0]))

    starts.append(len(egui))
    codes = np.array(codes)
    sf = raw.info.get('sfreq')
    starts = np.array(starts) / sf
    durations = starts[1:] - starts[:-1]
    starts = starts[:-1]

    raw.set_annotations(mne.Annotations(onset=starts, duration=durations, description=codes))


def raw_from_mat(file):
    mat = loadmat(os.path.join(data_dir, file))

    sampling_freq = mat["o"][0][0][2][0][0]
    n_samples = mat["o"][0][0][3][0][0]
    ch_names = [element[0][0] for element in mat["o"][0][0][6]]

    df = pd.DataFrame(mat["o"][0][0][5], columns=ch_names)
    df = df.drop(columns=["X5"])
    df = df.T
    ch_names.remove("X5")

    ch_types = ['eeg'] * 21
    info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
    raw = mne.io.RawArray(df.to_numpy(), info)

    montage = make_standard_montage("standard_prefixed")
    raw.set_montage(montage)

    raw.load_data().set_eeg_reference(ref_channels='average')
    annotations_from_eGUI(raw, mat["o"][0][0][4])
    return raw


def filter_raw(raw):
    return raw.load_data().filter(0.1, 30, method="fir", phase="zero-double")

In [5]:
raw_NoMT = [raw_from_mat(file) for file in data_files if "NoMT" in file]
raw_FREEFORM = [raw_from_mat(file) for file in data_files if "FREEFORM" in file]

Creating RawArray with float64 data, n_channels=21, n_times=664400
    Range : 0 ... 664399 =      0.000 ...  3321.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=664600
    Range : 0 ... 664599 =      0.000 ...  3322.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=662400
    Range : 0 ... 662399 =      0.000 ...  3311.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=21, n_times=667600
    Range : 0 ... 667599 =      0.000 ...  3337.995 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Creating RawArray with float64 d

In [6]:
def get_epochs(raw):
    metadata_tmin, metadata_tmax = -1, 1

    all_events, all_event_id = mne.events_from_annotations(raw)
    metadata, events, event_id = mne.epochs.make_metadata(
        events=all_events,
        event_id=all_event_id,
        tmin=metadata_tmin,
        tmax=metadata_tmax,
        sfreq=raw.info["sfreq"],
    )
    return mne.Epochs(raw, events, event_id, reject_by_annotation=True)


In [7]:
epochs_NoMT = [get_epochs(file) for file in raw_NoMT]
epochs_FREEFORM = [get_epochs(file) for file in raw_FREEFORM]

Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1931 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1919 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1925 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1935 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used

In [8]:
epochs_NoMT = [get_epochs(file) for file in raw_NoMT]
epochs_FREEFORM = [get_epochs(file) for file in raw_FREEFORM]


Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1931 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1919 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1925 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used Annotations descriptions: ['0', '1', '2', '3', '4', '5', '6', '91', '92', '99']
Not setting metadata
1935 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Used

In [9]:
epochs_NoMT[0].get_data().max()

Using data from preloaded Raw for 1931 events and 141 original time points ...
1 bad epochs dropped


865.3172938443672

In [10]:
epochs_FREEFORM[0].get_data().max()

Using data from preloaded Raw for 1481 events and 141 original time points ...
1 bad epochs dropped


158.92792102206735

In [11]:
epochs_data_NOMT = [file.get_data() for file in epochs_NoMT]
epochs_data_FREEFORM = [file.get_data() for file in epochs_FREEFORM]

Using data from preloaded Raw for 1930 events and 141 original time points ...
Using data from preloaded Raw for 1919 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1925 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1935 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1935 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1935 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1933 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1480 events and 141 original time points ...
Using data from preloaded Raw for 1383 events and 141 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1409 events and 141 original time points ...
1 bad epochs dropped


In [12]:
stacked_NOMT = np.vstack(epochs_data_NOMT)
stacked_FREEFORM = np.vstack(epochs_data_FREEFORM)

In [13]:
stacked_NOMT.shape

(13506, 21, 141)

In [14]:
stacked_FREEFORM.shape

(4270, 21, 141)

In [15]:
np.random.shuffle(stacked_NOMT)
np.random.shuffle(stacked_FREEFORM)

In [31]:
X_nomt_train = stacked_NOMT[:12000]
X_nomt_test = stacked_NOMT[12000:]

In [32]:
X_free = stacked_FREEFORM

In [33]:
# make Freeform test set same length as NoMT
idy = random.sample(range(0, len(X_free)), X_nomt_test.shape[0])
X_free_test = X_free[idy]

In [34]:
X_free_test.shape

(1506, 21, 141)

In [35]:
layer = layers.Normalization()
layer1 = layers.Normalization()
layer.adapt(X_nomt_train.astype(float))
layer1.adapt(X_free_test.astype(float))

print(X_nomt_train)
print(np.max(X_nomt_train))
print(np.max(X_free_test))
print(np.max(layer(X_nomt_train)))
print(np.max(layer1(X_free_test)))

[[[-9.27270616e-01 -1.61774681e+00 -2.84679443e+00 ... -6.65965157e+00
   -3.15774681e+00 -5.47393728e+00]
  [-9.88483159e+00 -1.14553078e+01 -4.06435540e+00 ... -9.32721254e+00
   -9.29530778e+00 -8.18149826e+00]
  [ 7.62241580e-01 -7.78234611e-01 -8.87282230e-01 ...  9.29860627e-01
    5.45176539e+00 -1.94442509e+00]
  ...
  [-1.00190476e+00 -2.23809524e-02 -1.63142857e+00 ... -3.92428571e+00
   -1.78238095e+00 -4.91857143e+00]
  [ 4.16387921e-01  9.45911731e-01  3.96864111e-01 ...  9.14006969e-01
    7.95911731e-01  4.97212544e-02]
  [ 1.29663182e+00  2.40615563e+00  1.66710801e+00 ...  6.53425087e+00
    6.28615563e+00  8.12996516e+00]]

 [[ 3.35360046e+00  2.31026713e+00  8.69169570e+00 ... -5.73401858e+00
   -2.73687573e+00 -6.93497096e+00]
  [ 3.43600465e-01  1.23026713e+00  3.49169570e+00 ... -5.51401858e+00
   -5.63687573e+00 -5.59497096e+00]
  [-7.12009292e-01  1.47465738e+00 -8.73914053e-01 ... -6.80962834e+00
   -7.59248548e+00 -7.18058072e+00]
  ...
  [ 9.70150987e-02 -3.0

In [36]:
def calc_accuracy(a, b, th):
    first = [1 if i < th else 0 for i in a]
    last = [1 if i > th else 0 for i in b]
    return sum(first + last) / len(first + last)

# Standard Autoencoder

In [37]:
latent_dim = 512
keras.backend.clear_session()


class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.Flatten(),
            layers.Dense(1024, activation='gelu'),
            layers.Dense(512, activation='gelu'),
            layers.Dense(64, activation='gelu'),
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(512, activation='gelu'),
            layers.Dense(1024, activation='gelu'),
            layers.Dense(21 * 141, activation='linear'),
            layers.Reshape((21, 141))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


autoencoder = Autoencoder(latent_dim)

In [38]:
opt = keras.optimizers.Adam(learning_rate=1e-3)
autoencoder.compile(optimizer=opt, loss=losses.MeanSquaredError())
autoencoder.fit(layer(X_nomt_train), layer(X_nomt_train),
                epochs=50,
                batch_size=64,
                shuffle=True,
                validation_data=(layer(X_nomt_test[:1000]), layer(X_nomt_test[:1000])))

Epoch 1/50
188/188 [==============================] - 7s 26ms/step - loss: 0.5549 - val_loss: 0.4079
Epoch 2/50
188/188 [==============================] - 5s 25ms/step - loss: 0.4655 - val_loss: 0.3988
Epoch 3/50
110/188 [================>.............] - ETA: 1s - loss: 0.3961

KeyboardInterrupt: 

In [ ]:
err = []
err2 = []
for i in X_nomt_train:
    # need to expand here because the flatten layer assumes that the first dimension is the number of samples
    i = np.expand_dims(i, axis=0)
    encoded = autoencoder.encoder(layer(i)).numpy()
    decoded = autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(i-decoded)).mean())
    err.append((np.square(layer(i) - decoded)).mean())
print("###################")

for j in X_free_test:
    j = np.expand_dims(j, axis=0)
    encoded = autoencoder.encoder(layer1(j)).numpy()
    decoded = autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(layer(j)-decoded)).mean())
    err2.append((np.square(layer1(j) - decoded)).mean())
print("##############")
print(np.array(err).mean())
print(np.array(err2).mean())


In [ ]:
calc_accuracy(err, err2, np.mean([np.array(err).mean(), np.array(err2).mean()]))


# Convolutional Autoencoder

In [39]:
X_nomt_train = X_nomt_train[..., np.newaxis]
X_nomt_test = X_nomt_test[..., np.newaxis]
X_free_test = X_free_test[..., np.newaxis]
X_nomt_train.shape

(12000, 21, 141, 1)

In [40]:
latent_dim = 512
keras.backend.clear_session()


class ConvAutoencoder(Model):
    def __init__(self, latent_dim):
        super(ConvAutoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(21, 141, 1)),
            layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
            layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


conv_autoencoder = ConvAutoencoder(latent_dim)


In [ ]:
opt = keras.optimizers.Adam(learning_rate=1e-3)
conv_autoencoder.compile(optimizer=opt, loss=losses.MeanSquaredError(), run_eagerly=True)
conv_autoencoder.fit(layer(X_nomt_train), layer(X_nomt_train),
                     epochs=50,
                     batch_size=32,
                     shuffle=True,
                     validation_data=(layer(X_nomt_test[:1000]), layer(X_nomt_test[:1000])))


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
err = []
err2 = []
for i in X_nomt_train:
    # need to expand here because the flatten layer assumes that the first dimension is the number of samples
    i = np.expand_dims(i, axis=0)
    encoded = conv_autoencoder.encoder(layer(i)).numpy()
    decoded = conv_autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(i-decoded)).mean())
    err.append((np.square(layer(i) - decoded)).mean())
print("###################")

for j in X_free_test:
    j = np.expand_dims(j, axis=0)
    encoded = conv_autoencoder.encoder(layer1(j)).numpy()
    decoded = conv_autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(layer(j)-decoded)).mean())
    err2.append((np.square(layer1(j) - decoded)).mean())
print("##############")
print(np.array(err).mean())
print(np.array(err2).mean())

In [ ]:
calc_accuracy(err, err2, np.mean([np.array(err).mean(), np.array(err2).mean()]))

In [ ]:
print(err[:10])

1,1,1,1,1,1,,0,1,1,1 9/10

In [ ]:
print(err2[:10])